# 初始化

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('../')  # 返回notebook的上一级目录
# sys.path.append('E:\GitHub\QA-abstract-and-reasoning')  # 效果同上

In [2]:
# 在google colab运行则执行以下代码
try:
    from google.colab import drive
    drive_path = '/content/drive'
    working_path = drive_path + "/My Drive/QA" # 工作路径
    drive.mount(drive_path)
    os.chdir(working_path)
    sys.path.append(working_path)  # 环境变量
    print("current working directory: ", os.getcwd())
    
    # %tensorflow_version 仅存在于 Colab
    %tensorflow_version 2.x
    print("run notebook in colab")
except:
    print("no colab")

no colab


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
np.set_printoptions(suppress=True)
from utils.plot import plot_attention
from utils.saveLoader import *
from utils.config import *
from layers import *
from preprocess import Preprocess
from gensim.models.word2vec import LineSentence, Word2Vec
import tensorflow as tf
# from model_layer import seq2seq_model
import time
tf.__version__

'2.0.0'

[限制gpu内存增长](https://tensorflow.google.cn/guide/gpu#limiting_gpu_memory_growth)

In [4]:
from utils.config_gpu import config_gpu
config_gpu()

1 Physical GPUs, 1 Logical GPUs


## 加载数据

In [5]:
train_x,train_y,test_x = load_train_dataset()  # 数据集
vocab,vocab_reversed = load_vocab(VOCAB_PAD)  # vocab
embedding_matrix = np.loadtxt(EMBEDDING_MATRIX_PAD)  # 预训练层

## 设置参数

In [6]:
params = {}
params["vocab_size"] = len(vocab)
params["max_enc_len"] = train_x.shape[1]  # 260
params["max_dec_len"] = train_y.shape[1]  # 33
params["embed_size"] = embedding_matrix.shape[1]
params["enc_units"] = 256
params["attn_units"] = 10
params["dec_units"] = params["enc_units"]
params["batch_size"] = 32
params["epochs"] = 2
print(params)

{'vocab_size': 32566, 'max_enc_len': 260, 'max_dec_len': 33, 'embed_size': 300, 'enc_units': 256, 'attn_units': 10, 'dec_units': 256, 'batch_size': 32, 'epochs': 2}


## 构建训练集

In [7]:
# 取部分数据进行训练
# sample_num=64
sample_num = train_x.shape[0]
dataset = tf.data.Dataset.from_tensor_slices((train_x[:sample_num], train_y[:sample_num])).shuffle(params["batch_size"]*2+1)
dataset = dataset.batch(params["batch_size"], drop_remainder=True)

In [8]:
steps_per_epoch = sample_num//params["batch_size"]

## 构建模型

In [9]:
from seq2seq import *
model=Seq2Seq(params)

## 保存点设置

In [10]:
from utils.config import CKPT_DIR, CKPT_PREFIX
from utils.saveLoader import del_all_files_of_dir
ckpt = tf.train.Checkpoint(Seq2Seq=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, CKPT_DIR, max_to_keep=5)

In [11]:
del_all_files_of_dir(CKPT_DIR)
ckpt.restore(ckpt_manager.latest_checkpoint)
print("Model restored")

there no files in this path
Model restored


## 训练

[SparseCategoricalCrossentropy](https://tensorflow.google.cn/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [12]:
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

pad_index=vocab['<PAD>']
unk_index=vocab['<UNK>']

def loss_function(real, pred):
    pad_mask = tf.math.equal(real, pad_index)
    unk_mask = tf.math.equal(real, unk_index)
    # <PAD> 和 <UNK> 的损失都不算
    mask = tf.math.logical_not(tf.math.logical_or(pad_mask,unk_mask))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
    # return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
# 调试train_step()
# inp, targ = next(iter(dataset))
# pad_index=vocab['<PAD>']
# unk_index=vocab['<UNK>']
# enc_output, enc_hidden = model.call_encoder(inp)
# dec_hidden = enc_hidden
# dec_input = tf.expand_dims([vocab['<START>']] * params["batch_size"], 1)
# predictions, _ = model(dec_input, dec_hidden, enc_output, targ)

In [13]:
@tf.function
def train_step(inp, targ):
    pad_index=vocab['<PAD>']
    unk_index=vocab['<UNK>']
    loss = 0
    
    with tf.GradientTape() as tape:
        # 1. 构建encoder
        enc_output, enc_hidden = model.call_encoder(inp)
        # 2. 复制
        dec_hidden = enc_hidden
        # 3. <START> * BATCH_SIZE 
        dec_input = tf.expand_dims([vocab['<START>']] * params["batch_size"], 1)
        
        # 逐个预测序列
        predictions, _ = model(dec_input, dec_hidden, enc_output, targ)
        
        batch_loss = loss_function(targ[:, 1:], predictions)

        variables = model.encoder.trainable_variables + model.decoder.trainable_variables+ model.attention.trainable_variables
    
        gradients = tape.gradient(batch_loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [ ]:
epochs = params["epochs"]
# 如果检查点存在，则恢复最新的检查点。
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')
    
for epoch in range(epochs):
    start = time.time()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch+1)):
        batch_loss = train_step(inp, targ)
        total_loss += batch_loss

        if batch % 1 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 5.3902
Epoch 1 Batch 1 Loss 5.6638
Epoch 1 Batch 2 Loss 5.2197
Epoch 1 Batch 3 Loss 5.5543
Epoch 1 Batch 4 Loss 5.8169
Epoch 1 Batch 5 Loss 4.5870
Epoch 1 Batch 6 Loss 5.0880
Epoch 1 Batch 7 Loss 4.8601
Epoch 1 Batch 8 Loss 4.3980
Epoch 1 Batch 9 Loss 3.2799
Epoch 1 Batch 10 Loss 5.2775
Epoch 1 Batch 11 Loss 3.6353
Epoch 1 Batch 12 Loss 2.7170
Epoch 1 Batch 13 Loss 2.6421
Epoch 1 Batch 14 Loss 2.1167
Epoch 1 Batch 15 Loss 2.9356
Epoch 1 Batch 16 Loss 2.4914
Epoch 1 Batch 17 Loss 2.3527
Epoch 1 Batch 18 Loss 2.4934
Epoch 1 Batch 19 Loss 3.1829
Epoch 1 Batch 20 Loss 3.3698
Epoch 1 Batch 21 Loss 3.2792
Epoch 1 Batch 22 Loss 3.1758
Epoch 1 Batch 23 Loss 3.3565
Epoch 1 Batch 24 Loss 2.8799
Epoch 1 Batch 25 Loss 2.5533
Epoch 1 Batch 26 Loss 2.5327
Epoch 1 Batch 27 Loss 3.1751
Epoch 1 Batch 28 Loss 3.1140
Epoch 1 Batch 29 Loss 2.9614
Epoch 1 Batch 30 Loss 3.0480
Epoch 1 Batch 31 Loss 3.4621
Epoch 1 Batch 32 Loss 3.3694
Epoch 1 Batch 33 Loss 2.5887
Epoch 1 Batch 34 Loss 1.

Epoch 1 Batch 277 Loss 2.5332
Epoch 1 Batch 278 Loss 2.7266
Epoch 1 Batch 279 Loss 2.2328
Epoch 1 Batch 280 Loss 2.3626
Epoch 1 Batch 281 Loss 2.8816
Epoch 1 Batch 282 Loss 2.6482
Epoch 1 Batch 283 Loss 2.3508
Epoch 1 Batch 284 Loss 2.4148
Epoch 1 Batch 285 Loss 2.1364
Epoch 1 Batch 286 Loss 1.7620
Epoch 1 Batch 287 Loss 2.5446
Epoch 1 Batch 288 Loss 1.9034
Epoch 1 Batch 289 Loss 2.3294
Epoch 1 Batch 290 Loss 2.4867
Epoch 1 Batch 291 Loss 2.5074
Epoch 1 Batch 292 Loss 1.7173
Epoch 1 Batch 293 Loss 2.1615
Epoch 1 Batch 294 Loss 2.1000
Epoch 1 Batch 295 Loss 2.4031
Epoch 1 Batch 296 Loss 1.9260
Epoch 1 Batch 297 Loss 2.8261
Epoch 1 Batch 298 Loss 2.3688
Epoch 1 Batch 299 Loss 2.1287
Epoch 1 Batch 300 Loss 2.3126
Epoch 1 Batch 301 Loss 2.2815
Epoch 1 Batch 302 Loss 1.7520
Epoch 1 Batch 303 Loss 2.2485
Epoch 1 Batch 304 Loss 1.6329
Epoch 1 Batch 305 Loss 2.2861
Epoch 1 Batch 306 Loss 2.0162
Epoch 1 Batch 307 Loss 2.9728
Epoch 1 Batch 308 Loss 2.4058
Epoch 1 Batch 309 Loss 2.5699
Epoch 1 Ba

## 载入模型

In [ ]:
# 如果检查点存在，则恢复最新的检查点。
ckpt.restore(ckpt_manager.latest_checkpoint)
print("Model restored")

In [ ]:
def evaluate(model,inputs):
    attention_plot = np.zeros((params["max_dec_len"], params["max_enc_len"]))
    
    inputs = tf.convert_to_tensor(inputs)

    result = ''
    
    hidden = [tf.zeros((1, params["enc_units"]))]
    enc_output, enc_hidden = model.encoder(inputs, hidden)

    dec_hidden = enc_hidden
    
    dec_input = tf.expand_dims([vocab['<START>']], 0)
    
    context_vector, _ = model.attention(dec_hidden, enc_output)

    for t in range(params["max_dec_len"]):
        
        context_vector, attention_weights = model.attention(dec_hidden, enc_output)
        
        predictions, dec_hidden = model.decoder(dec_input,
                                         dec_hidden,
                                         enc_output,
                                         context_vector)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        
        attention_plot[t] = attention_weights.numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()

        result += vocab_reversed[predicted_id] + ' '
        if vocab_reversed[predicted_id] == '<STOP>':
            return result, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, attention_plot

In [ ]:
def translate(sentence):
    st = preproc.sentence_proc(sentence)
    sentence = preproc.sentence_proc_eval(sentence,params["max_enc_len"]-2,vocab)
    result, attention_plot = evaluate(model,sentence)

    print('Input: %s' % (st))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(st.split(' '))]
    plot_attention(attention_plot, st.split(' '), result.split(' '))

In [ ]:
sentence = '方向机重，助力泵，方向机都换了还是一样'
preproc = Preprocess()
preproc.sentence_proc(sentence)

In [ ]:
# import matplotlib as mpl
# mpl.rcParams['font.family'] = 'STSong'  # 显示中文

In [ ]:
translate(sentence)

In [ ]:
# 下半部分
assert False

In [ ]:
def batch_predict(inps):
    # 判断输入长度
    batch_size=len(inps)
    # 开辟结果存储list
    preidicts=[''] * batch_size
    
    inps = tf.convert_to_tensor(inps)
    # 0. 初始化隐藏层输入
    hidden = [tf.zeros((batch_size, params["enc_units"]))]
    # 1. 构建encoder
    enc_output, enc_hidden = model.encoder(inps, hidden)
    # 2. 复制
    dec_hidden = enc_hidden
    # 3. <START> * BATCH_SIZE 
    dec_input = tf.expand_dims([vocab['<START>']] * batch_size, 1)
    
    context_vector, _ = model.attention(dec_hidden, enc_output)
    # Teacher forcing - feeding the target as the next input
    for t in range(params["max_dec_len"]):
        # 计算上下文
        context_vector, attention_weights = model.attention(dec_hidden, enc_output)
        # 单步预测
        predictions, dec_hidden = model.decoder(dec_input,
                                         dec_hidden,
                                         enc_output,
                                         context_vector)
        
        # id转换 贪婪搜索
        predicted_ids = tf.argmax(predictions,axis=1).numpy()
        
        
        for index,predicted_id in enumerate(predicted_ids):
            preidicts[index]+= vocab_reversed[predicted_id] + ' '
        
        # using teacher forcing
        dec_input = tf.expand_dims(predicted_ids, 1)

    results=[]
    for preidict in preidicts:
        # 去掉句子前后空格
        preidict=preidict.strip()
        # 句子小于max len就结束了 截断
        if '<STOP>' in preidict:
            # 截断stop
            preidict=preidict[:preidict.index('<STOP>')]
        # 保存结果
        results.append(preidict)
    return results

In [ ]:
# 测试代码
# ds = iter(dataset)
# x,y = ds.next()
# batch_predict(x)

In [ ]:
from tqdm import tqdm
import math

In [ ]:
def model_predict(data_X, batch_size):
    # 存储结果
    results=[]
    # 样本数量
    sample_size=len(data_X)
    # batch 操作轮数 math.ceil向上取整 小数 +1
    # 因为最后一个batch可能不足一个batch size 大小 ,但是依然需要计算  
    steps_epoch = math.ceil(sample_size/batch_size)
    # [0,steps_epoch)
    for i in tqdm(range(steps_epoch)):
        batch_data = data_X[i*batch_size:(i+1)*batch_size]
        results+=batch_predict(batch_data)
    return results

In [ ]:
%%time
results=model_predict(train_x[:sample_num+1],batch_size=4)